<a href="https://colab.research.google.com/github/simecek/PseudoDNA_Generator/blob/master/data/Random_Intron_Seqs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate Random Intron Sequences

## Setup

Installation for colab environment.

In [ ]:
!pip install biopython pyensembl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 537.2/537.2 kB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 763.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 3.1 MB/s eta 0:00:00
  Created wheel for memoized-property: filename=memoized_property-1.0.3-py2.py3-none-any.whl size=4184 sha256=8118112841043e303becdd51f3be92e6316ab01e81d599a889fdff30c146085f
  Stored in

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pyensembl install --release 97 --species human

2024-05-13 19:14:59,382 - pyensembl.shell - INFO - Running 'install' for EnsemblRelease(release=97, species='homo_sapiens')
2024-05-13 19:14:59,382 - pyensembl.download_cache - INFO - Fetching /root/.cache/pyensembl/GRCh38/ensembl97/Homo_sapiens.GRCh38.97.gtf.gz from URL https://ftp.ensembl.org/pub/release-97/gtf/homo_sapiens/Homo_sapiens.GRCh38.97.gtf.gz
2024-05-13 19:14:59,382 - datacache.download - INFO - Downloading https://ftp.ensembl.org/pub/release-97/gtf/homo_sapiens/Homo_sapiens.GRCh38.97.gtf.gz to /root/.cache/pyensembl/GRCh38/ensembl97/Homo_sapiens.GRCh38.97.gtf.gz
2024-05-13 19:15:03,485 - pyensembl.download_cache - INFO - Fetching /root/.cache/pyensembl/GRCh38/ensembl97/Homo_sapiens.GRCh38.cdna.all.fa.gz from URL https://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz
2024-05-13 19:15:03,486 - datacache.download - INFO - Downloading https://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz to

In [ ]:
import pandas as pd
import numpy as np
import gzip
from tqdm.notebook import tqdm

from Bio import SeqIO   # for reading fasta files
from pyensembl import EnsemblRelease   # to get the gene list
import random

ENSEMBL_RELEASE = 97
DNA_TOPLEVEL_FASTA_PATH = "/content/drive/My Drive/data/ensembl/Homo_sapiens.GRCh38.dna.toplevel.fa.gz"

# to generate random sequences
N = 20000    # how many
K = [random.randint(200, 500) for _ in range(20000)]
OUTPUT_FILE = '/content/drive/My Drive/data/random/introns.csv'   # where to save them

CHRS = [str(chr) for chr in range(1,23)] + ['X', 'Y', 'MT']

## Get exon list

In [ ]:
# release 97 uses human reference genome GRCh38
data = EnsemblRelease(ENSEMBL_RELEASE)

In [ ]:
human_exons = data.exon_ids()
len(human_exons)

745513

In [ ]:
human_exons[0], data.exon_by_id(human_exons[0])


('ENSE00000327880',
 Exon(exon_id='ENSE00000327880', gene_id='ENSG00000009780', gene_name='FAM76A', contig='1', start=27732603, end=27732657, strand='+'))

In [ ]:
exons_full_info  = [data.exon_by_id(exon) for exon in human_exons]

In [ ]:
human_exon_tuples = [(x.exon_id, x.gene_id, x.gene_name, x.contig, x.start, x.end, x.strand) for x in exons_full_info]
human_exon_table = pd.DataFrame.from_records(human_exon_tuples, columns=["id", "gene_id", "gene_symbol", "chr", "start", "end", "strand"])
assert all(human_exon_table.start <= human_exon_table.end)

human_exon_table['exon_noneverlaping_id'] = 0
human_exon_table = human_exon_table.sort_values(['gene_id', 'start', 'end'])
human_exon_table.head()

,id,gene_id,gene_symbol,chr,start,end,strand,exon_noneverlaping_id
634617,ENSE00003730948,ENSG00000000003,TSPAN6,X,100627109,100629986,-,0
79011,ENSE00001459322,ENSG00000000003,TSPAN6,X,100628670,100629986,-,0
14691,ENSE00000868868,ENSG00000000003,TSPAN6,X,100630759,100630866,-,0
634956,ENSE00003731560,ENSG00000000003,TSPAN6,X,100632063,100632068,-,0
197,ENSE00000401072,ENSG00000000003,TSPAN6,X,100632485,100632568,-,0


In [ ]:
human_exon_table_grouped = human_exon_table.groupby('gene_id')
human_exon_table_grouped.groups

{'ENSG00000000003': [634617, 79011, 14691, 634956, 197, 191550, 502849, 586420, 213112, 501640, 586426, 583014, 598849, 469616, 589333, 635984, 209012, 194362, 198020, 182183], 'ENSG00000000005': [79026, 196, 239672, 2936, 463117, 570997, 206526, 2937, 14690, 79022], 'ENSG00000000419': [449306, 520614, 176662, 95469, 506965, 662691, 2159, 213338, 492329, 498730, 77952, 434496, 491847, 462698, 571216, 235382, 492125, 556961, 465244, 596259, 580994, 597235, 96528, 77959, 477543, 534873, 211038], 'ENSG00000000457': [619880, 75286, 9893, 142436, 58990, 11416, 207713, 506024, 612718, 515827, 553265, 454640, 614570, 11415, 182566, 478999, 533617, 517724, 564784, 464117, 535860, 531090, 556279, 75285, 545176, 584972, 706936, 178032, 26825, 156818], 'ENSG00000000460': [224869, 184911, 174296, 200542, 216780, 179020, 70056, 205308, 523949, 63977, 71133, 624887, 63975, 471217, 536577, 606218, 663316, 453557, 646477, 667265, 456100, 532659, 460305, 484847, 212799, 227100, 186422, 458247, 475083, 

In [ ]:
human_exon_table_grouped.get_group('ENSG00000066827')

,id,gene_id,gene_symbol,chr,start,end,strand,exon_noneverlaping_id
282639,ENSE00002128386,ENSG00000066827,ZFAT,8,134477788,134478721,-,0
274686,ENSE00002100849,ENSG00000066827,ZFAT,8,134477792,134478701,-,0
491833,ENSE00003540302,ENSG00000066827,ZFAT,8,134477792,134478721,-,0
591972,ENSE00003665707,ENSG00000066827,ZFAT,8,134477792,134478721,-,0
540808,ENSE00003601450,ENSG00000066827,ZFAT,8,134478025,134478721,-,0
...,...,...,...,...,...,...,...,...
279775,ENSE00002118161,ENSG00000066827,ZFAT,8,134696431,134696558,-,0
281171,ENSE00002123129,ENSG00000066827,ZFAT,8,134712845,134712962,-,0
698989,ENSE00003841474,ENSG00000066827,ZFAT,8,134712845,134713031,-,0
92792,ENSE00001535723,ENSG00000066827,ZFAT,8,134712845,134713038,-,0


In [ ]:
def get_introns(df):
  if df.shape[0] <=1:
    return pd.DataFrame({'gene_id': [], 'chr': [], 'start': [], 'end': [], 'length': []})
  else:
    candidates = pd.DataFrame({'gene_id': df.gene_id.values[:-1], 'chr': df.chr.values[:-1], 'start': df.end.values[:-1]+1, 'end': df.start.values[1:]-1})
    candidates['length'] = candidates['end'] - candidates['start'] + 1
    return(candidates[candidates.length > 500])

get_introns(human_exon_table_grouped.get_group('ENSG00000066827'))

,gene_id,chr,start,end,length
7,ENSG00000066827,8,134478722,134509618,30897
12,ENSG00000066827,8,134510183,134510695,513
13,ENSG00000066827,8,134511083,134512474,1392
16,ENSG00000066827,8,134512602,134520882,8281
18,ENSG00000066827,8,134521002,134532833,11832
20,ENSG00000066827,8,134532973,134564986,32014
24,ENSG00000066827,8,134565672,134583831,18160
27,ENSG00000066827,8,134584006,134588245,4240
29,ENSG00000066827,8,134588396,134590267,1872
31,ENSG00000066827,8,134590356,134594806,4451


In [ ]:
human_introns = human_exon_table_grouped.apply(get_introns)
human_introns

gene_id chr        start          end  length
gene_id                                                                  
ENSG00000000003 1   ENSG00000000003   X  100629987.0  100630758.0   772.0
                2   ENSG00000000003   X  100630867.0  100632062.0  1196.0
                5   ENSG00000000003   X  100632569.0  100633404.0   836.0
                10  ENSG00000000003   X  100634030.0  100635177.0  1148.0
                18  ENSG00000000003   X  100637105.0  100639944.0  2840.0
...                             ...  ..          ...          ...     ...
ENSG00000288110 0   ENSG00000288110   8    4496495.0    4497861.0  1367.0
                1   ENSG00000288110   8    4498056.0    4499121.0  1066.0
                2   ENSG00000288110   8    4499222.0    4501372.0  2151.0
ENSG00000288111 0   ENSG00000288111   3  130181410.0  130182746.0  1337.0
                1   ENSG00000288111   3  130182919.0  130188596.0  5678.0

[219137 rows x 5 columns]

In [ ]:
human_introns = human_introns.reset_index(drop=True)
human_introns['start'] = human_introns['start'].astype('int')
human_introns['end'] = human_introns['end'].astype('int')
human_introns['length'] = human_introns['length'].astype('int')
human_introns

,gene_id,chr,start,end,length
0,ENSG00000000003,X,100629987,100630758,772
1,ENSG00000000003,X,100630867,100632062,1196
2,ENSG00000000003,X,100632569,100633404,836
3,ENSG00000000003,X,100634030,100635177,1148
4,ENSG00000000003,X,100637105,100639944,2840
...,...,...,...,...,...
219132,ENSG00000288110,8,4496495,4497861,1367
219133,ENSG00000288110,8,4498056,4499121,1066
219134,ENSG00000288110,8,4499222,4501372,2151
219135,ENSG00000288111,3,130181410,130182746,1337


In [ ]:
selected_regions = human_introns.copy()
sample_regions = selected_regions.sample(N)
sample_regions['random_pos'] = [np.random.randint(c_len - K[i]) for i, c_len in enumerate(sample_regions.length)]
sample_regions['random_start'] = sample_regions.start + sample_regions.random_pos
sample_regions['random_end'] = sample_regions['random_start'] + K - 1
sample_regions.head()

,gene_id,chr,start,end,length,random_pos,random_start,random_end
132911,ENSG00000176406,8,103922009,103927841,5833,5050,103927059,103927394
122706,ENSG00000169213,1,51977118,51990551,13434,2096,51979214,51979608
198485,ENSG00000259222,15,69091115,69097583,6469,3758,69094873,69095337
25044,ENSG00000092929,17,75828984,75829661,678,360,75829344,75829645
103916,ENSG00000158125,2,31341395,31342182,788,272,31341667,31341937


## Random exon selection

In [ ]:
sample_regions.shape

(20000, 8)

## Get actual genomic sequences

In [ ]:
seqs = sample_regions[['gene_id', 'chr', 'random_start', 'random_end']].copy().reset_index(drop=True)
seqs['seq'] = ''

In [ ]:
seqs.head()

,gene_id,chr,random_start,random_end,seq
0,ENSG00000176406,8,103927059,103927394,
1,ENSG00000169213,1,51979214,51979608,
2,ENSG00000259222,15,69094873,69095337,
3,ENSG00000092929,17,75829344,75829645,
4,ENSG00000158125,2,31341667,31341937,


In [ ]:
def which(self):
    try:
        self = list(iter(self))
    except TypeError as e:
        raise Exception("""'which' method can only be applied to iterables.
        {}""".format(str(e)))
    indices = [i for i, x in enumerate(self) if bool(x) == True]
    return(indices)

with gzip.open(DNA_TOPLEVEL_FASTA_PATH, "rt") as handle:
    for record in tqdm(SeqIO.parse(handle, "fasta"), total=24):
        sel_seqs = which(seqs.chr == record.id)
        for i in sel_seqs:
            seqs.loc[i, "seq"] = str(record.seq[(seqs.random_start[i]-1):seqs.random_end[i]])

        if record.id == "MT":
            # stop, do not read small contigs
            break

  0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
seqs.head()

,gene_id,chr,random_start,random_end,seq
0,ENSG00000176406,8,103927059,103927394,GACATACCATGAGGCAATAAACTTACAATAAACTGTCAATAAAGGC...
1,ENSG00000169213,1,51979214,51979608,CATGGTACTAGGTGCAAAGGCTGTTGCAAGAACTTGCCATGTAAAG...
2,ENSG00000259222,15,69094873,69095337,GGGTTCACAGCTGGACTGGGAGCAGCAGACATACAGGCCGTAGGCA...
3,ENSG00000092929,17,75829344,75829645,GGAGTGCAGTGGCGCCATCTTGGCTCACTACAACCTCCGCCTCCCG...
4,ENSG00000158125,2,31341667,31341937,GAGTAGTCAGATAGGGTCCCAGAAACCTGGGGCTTTGAATCTGAGT...


In [ ]:
len(seqs.seq.values[0]), seqs.seq.values[0]

(336,
 'GACATACCATGAGGCAATAAACTTACAATAAACTGTCAATAAAGGCTTGGAAAAGTATAACGAACTGTATAAGCAAAAAGAAACAAGATATTGTCATTAATAGGTGTGATTAGCAGTAAGCAGTGTTATCTTGAGTGATACATAATAAATATCTGATGATGGTGTTTCAATACTAGAACCTGATAGGGATTCACCAAGTATGAATCAACTAATTAGTCCAACGTATTTCTGCACACTTTTATTTTAATGTGTTTATTCTTTTAAGAAATTCTAAAGCCAACTTGGCTAATAGTTTTTGGACTTTGGGTAGACATACTTTGGAGAAGTCACACAATT')

In [ ]:
sum(seqs.seq.str.contains('N'))

2

## Save generated sequences to file

In [ ]:
output = seqs[~seqs.seq.str.contains('N')]
output.shape

(19998, 5)

In [ ]:
output[:N].to_csv('/content/drive/My Drive/genomic_data/introns.csv', index=False)